# Research Space Collaboration Network

This script generates a plot showing each professors position in the research space and then connects them with edges.
The edges width is determined by how often they collaborated.

In [8]:
import os
import ast
import imageio
import numpy as np
import pandas as pd
from tqdm import tqdm
from tueplots import bundles
import matplotlib.pyplot as plt
from IPython.display import Image
from tueplots.constants.color import rgb
import matplotlib.lines as lines

plt.rcParams.update(bundles.beamer_moml())
plt.rcParams.update({"figure.dpi": 300, 'font.family': 'Times New Roman'})

In [9]:
df = pd.read_csv("./../dat/ProjectedPublications.csv")
unique_df = df.drop_duplicates(subset='Title', keep='first')

years = range(df['Year'].min(), df['Year'].max() + 1)
professors = df["Professor"].unique()

research_fields = [
    'Computer Vision',
    'Robotics',
    'Bioinformatics and Computational Biology',
    'Human-Computer Interaction',
    'Design Automation of Microelectronic Systems (EDA)',
    'Visualization', 
    'Web Information Retrieval',
    'Natural Language Processing',
    'Machine Learning', 
]

sorted_research_fields = ["Machine Learning", "Computer Vision", "Robotics", "Bioinformatics and Computational Biology", "Human-Computer Interaction", "Visualization", "Design Automation of Microelectronic Systems (EDA)", "Web Information Retrieval", "Natural Language Processing"]
sorted_colors = [rgb.tue_darkgreen, rgb.pn_orange, rgb.tue_darkblue, rgb.tue_lightblue, rgb.tue_brown, rgb.tue_blue, rgb.tue_green, rgb.tue_dark, rgb.tue_red]
colors = dict(zip(sorted_research_fields, sorted_colors))

def format_name(name):
    parts = name.split()
    formatted_name = parts[0][0].upper() + ". " + parts[-1]
    return formatted_name

In [14]:
professors_colleagues_coauthors = {prof: {} for prof in professors}

for _, row in df.iterrows():
    prof = row.iloc[0]
    for coauthor in row.iloc[4].split(';'):
        if (coauthor != prof):
            if coauthor in professors_colleagues_coauthors[prof]:
                professors_colleagues_coauthors[prof][coauthor] +=1
            else:
                professors_colleagues_coauthors[prof][coauthor] = 1                 

for prof in professors_colleagues_coauthors:
    professors_colleagues_coauthors[prof] = dict(sorted(professors_colleagues_coauthors[prof].items(), key=lambda item: item[1], reverse=True))

for year in tqdm(years, desc="Creating Plot for Each Year"):
    professor_positions = {prof: [] for prof in professors}
    year_df = df[df['Year'] <= year]
    unique_year_df = unique_df[unique_df['Year'] <= year]

    fig, ax = plt.subplots()
    fig.set_size_inches(fig.get_size_inches()[0], fig.get_size_inches()[0])

    # Professors
    for professor in professors:
        professor_df = year_df[year_df['Professor'] == professor]
        if len(professor_df) != 0:
            professor_x, professor_y = professor_df[['x', 'y']].mean()
            ax.scatter(professor_x, professor_y, color=rgb.tue_dark, alpha=0.7, s=20, zorder=2)
            y, va = 0.3, 'bottom'
            if professor == 'Philipp Berens':
                y, va = -y, 'top'
            ax.text(professor_x, professor_y + y, format_name(professor), ha='center', va=va, fontsize=5.5, zorder=4)
            professor_positions[professor] = [professor_x, professor_y]

    # Drawing connection lines
    for prof1 in professors:
        for prof2 in professors:
            prof1_pos, prof2_pos = professor_positions[prof1], professor_positions[prof2]
            if (len(prof1_pos) != 0) and (len(prof2_pos) != 0) and prof2 in list(professors_colleagues_coauthors[prof1].keys()):
                line = lines.Line2D([prof1_pos[0],prof2_pos[0]], [prof1_pos[1], prof2_pos[1]], color='gray', linewidth=professors_colleagues_coauthors[prof1][prof2]/3, alpha=0.2)
                ax.add_line(line)

    ax.set_xlim([-80, 80])
    ax.set_ylim([-80, 80])

    ax.set_xticks([])
    ax.set_yticks([])
    ax.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False)

    ax.set_xlabel('')
    ax.set_ylabel('')
    
    ax.set_title(f'Research Space Network (Until Year {year})')

   # if legend:
    #    ax.legend(title="Research Fields", loc="lower right", framealpha=1, facecolor="white", frameon=True, fontsize='x-small')

    plt.close()

    fig.savefig(f'./../plt/ResearchSpaceNetwork/{year}.png')

Creating Plot for Each Year: 100%|█████████████████████████████████████████████████████| 35/35 [00:11<00:00,  2.93it/s]
